In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import simps
from ed import *
from dos import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_G0_semicirc(omFs, mu, t):
    from dos import semicircle
    from scipy.integrate import simps
    ws = np.linspace(-2*t,2*t,1000)
    sc_dos = semicircle(ws,t)
    G0 = np.zeros(len(omFs),dtype=complex)
    for iomF, omF in enumerate(omFs):
        y = sc_dos/(1j*omF+mu-ws)
        G0[iomF] = simps(y, ws)
    return G0

In [3]:
def get_G0_aim(omFs, V, eb, ed, mu):
    G0_aim = np.zeros(len(omFs),dtype=complex)
    for iomF, omF in enumerate(omFs):
        Delta = 0.0
        for ib in range(eb.shape[0]):
            Delta += (V[ib]*np.conj(V[ib])).real/(1j*omF-eb[ib])
        G0_aim[iomF] = 1./(1j*omF - ed + mu - Delta)
    return G0_aim

def get_G0w_aim(ws, eta, V, eb, ed, mu):
    G0_aim = np.zeros((len(ws)),dtype=complex)
    for iw, w in enumerate(ws):
        Delta = 0.0
        for ib in range(eb.shape[0]):
            Delta += (V[ib]*np.conj(V[ib])).real/(w + 1j*eta - eb[ib])
        G0_aim[iw] = (1./(w + 1j*eta - ed + mu - Delta))[0]
    return G0_aim

In [4]:
def cost_func(x, *args):
    omFs, ed, mu, G0, Nb, Nmax = args
    V = x[:Nb] + 1j*x[Nb:2*Nb]
    eb = x[2*Nb:3*Nb]
    G0_aim = get_G0_aim(omFs, V, eb, ed ,mu)
    cost = 0.0
    for iomF in range(Nmax):
        diff = 1./G0_aim[iomF] - 1./G0[iomF]
        cost += (np.conj(diff)*diff).real/omFs[iomF]**1
    return cost 

In [5]:
np.random.seed(1234)
np.set_printoptions(suppress=True, precision=8)

# initial density of state
T = 0.005
t = 0.5
mu = 0.0
Nb = 3
Nmax = 100
NomF = Nmax
omFs = (2*np.arange(NomF)+1)*np.pi*T
G0 = get_G0_semicirc(omFs, mu, t)

no = 2*(1+Nb)
print('no=',no)
FH_list = build_fermion_op(no)
print(len(FH_list))
FH_list_dense = [np.array(FH.todense(),dtype=complex) for FH in FH_list]

ebs = np.random.uniform(-1,1,Nb)
Vrs = np.random.uniform(-0.5,0.5,Nb)
Vis = np.zeros((Nb))
x0 = np.hstack((Vrs,Vis,ebs))
mix = 0.3
maxit = 500

Us = np.arange(0.0,2.1,0.1)[::-1]
  
doccs = []
for U in Us:
    it = 0
    diff = 1e20
    while diff > 5e-5 and it<maxit:
        print("-------------------------------------- it=%d U=%.2f --------------------------------------"%(it, U))

        # fit V eb
        args = omFs, 0.0, mu, G0, Nb, Nmax
        result = minimize(cost_func,x0,args=args, method='L-BFGS-B', options={'gtol': 1e-2, 'eps': 1e-12})
        print("GA root convergence message---------------------------------")
        print("sucess=",result.success)
        print(result.message)
        V = result.x[:Nb] + 1j*result.x[Nb:2*Nb]
        eb = result.x[2*Nb:3*Nb]
        x0 = result.x
        print('V=',V)
        print('eb=', eb)
    
        # ED part
        h1 = np.array([[-U/2+mu, 0.0],
                        [ 0.0,-U/2+mu]])
        eb = np.kron(eb,np.ones((2)))
        V = np.kron(V,np.eye(2)).T
        #
        V2E = np.zeros((2,2,2,2))
        V2E[1,1,0,0] = U
        V2E[0,0,1,1] = U
        print('V(ED)=',V)
        print('eb(ED)=', eb)

        dm, evals, evecs, docc = solve_Hemb_thermal(T, h1, V, eb, V2E, FH_list, verbose=0)
        
        print('dm=')
        print(dm.real)
        print('trace(dm)=',np.trace(dm).real)
        print('docc=', docc.real)
    
        GomF = compute_GomF_thermal(T, omFs, evals, evecs, FH_list_dense)
      
        # update G0
        G0_new = 1./(1j*omFs + mu - t**2*GomF)
    
        diff = np.sum(np.abs(G0_new  -G0))
        print('diff=', diff)
        it += 1
        G0 = (1-mix)*G0 + mix*G0_new

    # real frequency quantities
    ws = np.linspace(-5,5,200) # frequency mesh
    eta = 0.1 #broadening factor
    Gw = compute_Gw_thermal(T, ws, eta, evals, evecs, FH_list_dense) # real frequency impurity Green's function
    G0w = get_G0w_aim(ws, eta, V[::2,::2], eb[::2], 0.0, mu) # real frequency non-interacting impurity Green's function
    Sigw = 1./G0w - 1./Gw # real frequency self-energy
    Glattw = compute_Glattw_semcircle(ws,eta,Sigw,mu) # real frequency lattice Green's function
    # Matsubara self-energy
    G0 = get_G0_aim(omFs, V[::2,::2], eb[::2], 0.0, mu) # matsubara frequency impurity Green's function
    Sig = 1./G0 - 1./GomF # Matsubara frequency self-energy
    Sig = 1./G0 - 1./GomF
    Z = 1./(1-(Sig[0].imag)/(omFs[0])) # quasiparticle weight estimate from Matsubara self-energy
    Glatt = compute_GlattomF_semcircle(omFs,Sig,mu) # real frequency lattice Green's function

    
    np.savetxt('Gimpw_U%.2f.dat'%(U),np.vstack((ws,Gw.real,Gw.imag)).T)
    np.savetxt('Glattw_U%.2f.dat'%(U),np.vstack((ws,Glattw.real,Glattw.imag)).T)
    np.savetxt('Gimp_U%.2f.dat'%(U), np.vstack((omFs,GomF.real,GomF.imag)).T)
    np.savetxt('Glatt_U%.2f.dat'%(U), np.vstack((omFs,Glatt.real,Glatt.imag)).T)
    np.savetxt('G0_U%.2f.dat'%(U), np.vstack((omFs,G0.real,G0.imag)).T)
    np.savetxt('Sig_U%.2f.dat'%(U), np.vstack((omFs,Sig.real,Sig.imag)).T)
    np.savetxt('Sigw_U%.2f.dat'%(U), np.vstack((ws,Sigw.real,Sigw.imag)).T)
    np.savetxt('x0_U%.2f.dat'%(U), x0)
    doccs.append(docc.real)
    print(U, dm[0,0].real, dm[1,1].real, docc.real, Z, diff)
    f = open('U_occ_docc_Z_diff_mott2metal.dat','a')
    print(U, dm[0,0].real, dm[1,1].real, docc.real, Z, diff, file=f)
    f.close()

no= 8
8
-------------------------------------- it=0 U=4.00 --------------------------------------
GA root convergence message---------------------------------
sucess= True
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
V= [ 0.26451211+0.j  0.26451413+0.j -0.07980677+0.j]
eb= [-0.15225325  0.15225749  0.00000006]
V(ED)= [[ 0.26451211+0.j  0.        +0.j]
 [ 0.        +0.j  0.26451211+0.j]
 [ 0.26451413+0.j  0.        +0.j]
 [ 0.        +0.j  0.26451413+0.j]
 [-0.07980677+0.j -0.        +0.j]
 [-0.        +0.j -0.07980677+0.j]]
eb(ED)= [-0.15225325 -0.15225325  0.15225749  0.15225749  0.00000006  0.00000006]
Egs= -2.4228525606682254
evals= [0.         0.01758063 0.01758063 0.01758097 0.01758097 0.02553739
 0.02553739 0.02553739 0.08179146 0.08179302]
dm=
[[ 0.49999996 -0.         -0.12750049 -0.         -0.12749977  0.
   0.13723702 -0.        ]
 [-0.          0.49999996 -0.         -0.12750049  0.         -0.12749977
  -0.          0.13723702]
 [-0.12750049 -0.          0.90336344 -0. 

dm=
[[ 0.50000135  0.         -0.04951625  0.         -0.04951127 -0.
   0.02499729 -0.        ]
 [ 0.          0.50000135  0.         -0.04951625 -0.         -0.04951127
   0.          0.02499729]
 [-0.04951625  0.          0.9935248   0.         -0.00000399 -0.
   0.00694282 -0.        ]
 [ 0.         -0.04951625  0.          0.9935248  -0.         -0.00000399
   0.          0.00694282]
 [-0.04951127 -0.         -0.00000399 -0.          0.0064752   0.
  -0.00694649  0.        ]
 [-0.         -0.04951127 -0.         -0.00000399  0.          0.0064752
   0.         -0.00694649]
 [ 0.02499729  0.          0.00694282  0.         -0.00694649  0.
   0.50024497  0.        ]
 [-0.          0.02499729 -0.          0.00694282  0.         -0.00694649
   0.          0.50024497]]
trace(dm)= 4.000492653854861
docc= 0.00913623489434557
diff= 28.586088532452678
-------------------------------------- it=6 U=4.00 --------------------------------------
GA root convergence message-----------------------

diff= 26.713773773912767
-------------------------------------- it=11 U=4.00 --------------------------------------
GA root convergence message---------------------------------
sucess= True
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
V= [ 0.29883398+0.j  0.29925344+0.j -0.01438505+0.j]
eb= [-1.28865287  1.29280686  0.00001063]
V(ED)= [[ 0.29883398+0.j  0.        +0.j]
 [ 0.        +0.j  0.29883398+0.j]
 [ 0.29925344+0.j  0.        +0.j]
 [ 0.        +0.j  0.29925344+0.j]
 [-0.01438505+0.j -0.        +0.j]
 [-0.        +0.j -0.01438505+0.j]]
eb(ED)= [-1.28865287 -1.28865287  1.29280686  1.29280686  0.00001063  0.00001063]
Egs= -4.6329023829855345
evals= [0.         0.00032133 0.00032133 0.00034261 0.00034261 0.00044266
 0.00044266 0.00044266 1.08293006 1.08298577]
dm=
[[ 0.49999946 -0.         -0.04679301 -0.         -0.04679629  0.
   0.00407165  0.        ]
 [-0.          0.49999946 -0.         -0.04679301  0.         -0.04679629
  -0.          0.00407165]
 [-0.04679301 -0.      

dm=
[[ 0.50000549 -0.         -0.04612339 -0.         -0.04608726  0.
   0.00176683 -0.        ]
 [-0.          0.50000549 -0.         -0.04612339  0.         -0.04608726
   0.          0.00176683]
 [-0.04612339 -0.          0.99564795 -0.         -0.00000795  0.
   0.0001713   0.        ]
 [-0.         -0.04612339 -0.          0.99564795  0.         -0.00000795
   0.          0.0001713 ]
 [-0.04608726  0.         -0.00000795  0.          0.00434535 -0.
  -0.00017157 -0.        ]
 [ 0.         -0.04608726  0.         -0.00000795 -0.          0.00434535
   0.         -0.00017157]
 [ 0.00176683  0.          0.0001713   0.         -0.00017157  0.
   0.40900991 -0.        ]
 [-0.          0.00176683  0.          0.0001713  -0.         -0.00017157
  -0.          0.40900991]]
trace(dm)= 3.8180174110961547
docc= 0.008189807863360817
diff= 7.184021285901561
-------------------------------------- it=17 U=4.00 --------------------------------------
GA root convergence message--------------------

diff= 1.7996536423563645
-------------------------------------- it=22 U=4.00 --------------------------------------
GA root convergence message---------------------------------
sucess= True
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
V= [ 0.31749804+0.j  0.31693825+0.j -0.00155803+0.j]
eb= [-1.56599356  1.56066836  0.00152082]
V(ED)= [[ 0.31749804+0.j  0.        +0.j]
 [ 0.        +0.j  0.31749804+0.j]
 [ 0.31693825+0.j  0.        +0.j]
 [ 0.        +0.j  0.31693825+0.j]
 [-0.00155803+0.j -0.        +0.j]
 [-0.        +0.j -0.00155803+0.j]]
eb(ED)= [-1.56599356 -1.56599356  1.56066836  1.56066836  0.00152082  0.00152082]
Egs= -5.189093957466249
evals= [0.         0.         0.00151697 0.00152211 0.00152211 0.00152211
 0.00304165 0.00304165 1.28008376 1.28160394]
dm=
[[ 0.50000144 -0.         -0.04549538 -0.         -0.04548666 -0.
   0.00042015 -0.        ]
 [-0.          0.50000144 -0.         -0.04549538  0.         -0.04548666
  -0.          0.00042015]
 [-0.04549538 -0.       

...

dm=
[[ 0.49999987 -0.         -0.24016765 -0.         -0.24016785  0.
   0.23343964  0.        ]
 [-0.          0.49999987 -0.         -0.24016765  0.         -0.24016785
   0.          0.23343964]
 [-0.24016765 -0.          0.80417382  0.         -0.00000206 -0.
   0.27362061  0.        ]
 [-0.         -0.24016765  0.          0.80417382  0.         -0.00000206
   0.          0.27362061]
 [-0.24016785  0.         -0.00000206  0.          0.19582878 -0.
  -0.27362273 -0.        ]
 [ 0.         -0.24016785 -0.         -0.00000206 -0.          0.19582878
  -0.         -0.27362273]
 [ 0.23343964  0.          0.27362061  0.         -0.27362273 -0.
   0.49999803 -0.        ]
 [ 0.          0.23343964  0.          0.27362061 -0.         -0.27362273
  -0.          0.49999803]]
trace(dm)= 4.000000985924448
docc= 0.11243158455190372
diff= 8.725163381587818e-05
-------------------------------------- it=32 U=1.50 --------------------------------------
GA root convergence message------------------

diff= 2.46037771664807
-------------------------------------- it=3 U=1.40 --------------------------------------
GA root convergence message---------------------------------
sucess= True
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
V= [ 0.22272844+0.j  0.22272965+0.j -0.07770837+0.j]
eb= [-0.12150667  0.12150563 -0.00000038]
V(ED)= [[ 0.22272844+0.j  0.        +0.j]
 [ 0.        +0.j  0.22272844+0.j]
 [ 0.22272965+0.j  0.        +0.j]
 [ 0.        +0.j  0.22272965+0.j]
 [-0.07770837+0.j -0.        +0.j]
 [-0.        +0.j -0.07770837+0.j]]
eb(ED)= [-0.12150667 -0.12150667  0.12150563  0.12150563 -0.00000038 -0.00000038]
Egs= -1.1920450530339826
evals= [0.         0.02587581 0.02587581 0.02587703 0.02587703 0.04657251
 0.04657251 0.04657251 0.05888819 0.05888955]
dm=
[[ 0.50000011  0.         -0.24752292  0.         -0.24752577 -0.
   0.23750251 -0.        ]
 [ 0.          0.50000011  0.         -0.24752292 -0.         -0.24752577
  -0.          0.23750251]
 [-0.24752292  0.         

dm=
[[ 0.50000043 -0.         -0.24836846 -0.         -0.24835869 -0.
   0.23716215 -0.        ]
 [-0.          0.50000043 -0.         -0.24836846 -0.         -0.24835869
  -0.          0.23716215]
 [-0.24836846 -0.          0.79794785 -0.          0.00001461  0.
   0.27763224 -0.        ]
 [-0.         -0.24836846 -0.          0.79794785  0.          0.00001461
  -0.          0.27763224]
 [-0.24835869 -0.          0.00001461  0.          0.20202439 -0.
  -0.27761803  0.        ]
 [-0.         -0.24835869  0.          0.00001461 -0.          0.20202439
   0.         -0.27761803]
 [ 0.23716215 -0.          0.27763224 -0.         -0.27761803  0.
   0.50002058  0.        ]
 [-0.          0.23716215 -0.          0.27763224  0.         -0.27761803
   0.          0.50002058]]
trace(dm)= 3.9999865077024266
docc= 0.12116221619441601
diff= 0.30036304987785606
-------------------------------------- it=9 U=1.40 --------------------------------------
GA root convergence message--------------------

diff= 0.05143958292931291
-------------------------------------- it=14 U=1.40 --------------------------------------
GA root convergence message---------------------------------
sucess= True
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
V= [ 0.22437092+0.j  0.22435867+0.j -0.07769052+0.j]
eb= [-0.1220696   0.12204398 -0.00000159]
V(ED)= [[ 0.22437092+0.j  0.        +0.j]
 [ 0.        +0.j  0.22437092+0.j]
 [ 0.22435867+0.j  0.        +0.j]
 [ 0.        +0.j  0.22435867+0.j]
 [-0.07769052+0.j -0.        +0.j]
 [-0.        +0.j -0.07769052+0.j]]
eb(ED)= [-0.1220696  -0.1220696   0.12204398  0.12204398 -0.00000159 -0.00000159]
Egs= -1.1959557066633115
evals= [0.         0.0258355  0.0258355  0.02584319 0.02584319 0.04654089
 0.04654089 0.04654089 0.05873362 0.05874305]
dm=
[[ 0.49999894 -0.         -0.24846844 -0.         -0.24847805  0.
   0.23711056  0.        ]
 [-0.          0.49999894 -0.         -0.24846844  0.         -0.24847805
  -0.          0.23711056]
 [-0.24846844 -0.     

KeyboardInterrupt: 